In [1]:
import pandas as pd
from tqdm import tqdm
import json

In [2]:
temp = pd.read_csv("result.csv")
temp = temp[temp["dir_ratio"] <= 0.5]
EXCLUDE=temp["dataset"].unique().tolist()

In [3]:
# AdaBoost_raw = pd.read_csv("result_mp_adjust_adaboost.csv")
# case0_raw = pd.read_csv("result_mp_adjust_case0.csv")
# custom_raw = pd.read_csv("result_mp_adjust_product.csv")

# merge
# df = pd.concat([AdaBoost_raw, case0_raw, custom_raw], ignore_index=True)
df = pd.read_csv("result_mp_adjust_wn.csv")
# remove invalid data
df = df[(df["dist_error"]!=0) | (df["dir_error"]!=0) | (df["dir_ratio"]!=0)]
df = df[~df["dataset"].isin(EXCLUDE)]

# select parameters that have `dist_ratio` > 0.9 on test_case0
acceptable = df[df["dataset"] == "../test_case0"]
acceptable = acceptable[acceptable["dir_ratio"] > 0.9]
acceptable = acceptable[[
    'model_name',
    'distance_frac_step',
    'clean_data',
    'optimized_mode_ratio',
    'butter_Wn']]
# acceptable = acceptable.drop_duplicates()
# acceptable = acceptable.values.tolist()

In [4]:
# compare by dir_ratio, test_case0 has a weight of 25, others have a weight of 1

# add weight
df["weight"] = 1
df.loc[df["dataset"] == "../test_case0", "weight"] = 25

# compare
selected_args_ratio = []
max_mean_dir_ratio = 0
no_weight_ratio=0
for args in tqdm(acceptable[['clean_data','optimized_mode_ratio','butter_Wn']].drop_duplicates().values.tolist()):
    temp = df[
        (df["clean_data"] == args[0]) &
        (df["optimized_mode_ratio"] == args[1]) &
        (df["butter_Wn"] == args[2])
    ]
    if len(temp) == 0:
        continue
    mean_dir_ratio = (temp["dir_ratio"] * temp["weight"]).sum() / temp["weight"].sum()
    if mean_dir_ratio > max_mean_dir_ratio:
        max_mean_dir_ratio = mean_dir_ratio
        selected_args_ratio = args
        no_weight_ratio = temp["dir_ratio"].mean()

100%|██████████| 48/48 [00:00<00:00, 610.08it/s]


In [5]:
print('Best args: '+json.dumps({
    "clean_data": selected_args_ratio[0],
    "optimized_mode_ratio": selected_args_ratio[1],
    "butter_Wn": selected_args_ratio[2]
}, indent=4))
print('Mean dir_ratio (with weight): '+ str(max_mean_dir_ratio))
print('Mean dir_ratio (without weight): '+ str(no_weight_ratio))
df[
    (df["clean_data"] == selected_args_ratio[0]) &
    (df["optimized_mode_ratio"] == selected_args_ratio[1]) &
    (df["butter_Wn"] == selected_args_ratio[2])
][["dataset", "dist_error", "dir_error", "dir_ratio", "weight"]].sort_values('dir_ratio', ascending=False).reset_index(drop=True)

Best args: {
    "clean_data": 4.0,
    "optimized_mode_ratio": 0.95,
    "butter_Wn": 0.0035
}
Mean dir_ratio (with weight): 0.7969390058417374
Mean dir_ratio (without weight): 0.6918891015119726


,dataset,dist_error,dir_error,dir_ratio,weight
0,Pocket-Walk/Pocket-Walk-09-001,73.460831,5.704716,0.938144,1
1,Pocket-Walk/Pocket-Walk-09-001,68.322989,5.704716,0.938144,1
2,Hand-Walk/Hand-Walk-02-001,14.075708,7.143046,0.928889,1
3,Hand-Walk/Hand-Walk-02-001,14.711121,7.143046,0.928889,1
4,../test_case0,30.032920,8.662983,0.926063,25
5,../test_case0,44.214368,8.662983,0.926063,25
6,Hand-Walk/Hand-Walk-09-001,35.989921,8.284083,0.922078,1
7,Hand-Walk/Hand-Walk-09-001,39.797751,8.284083,0.922078,1
8,Hand-Walk/Hand-Walk-02-008,22.224224,8.734746,0.855037,1
9,Hand-Walk/Hand-Walk-02-008,17.926071,8.734746,0.855037,1


In [6]:
# compare by dist_error, test_case0 has a weight of 25, others have a weight of 1
selected_args = []
min_mean_dist_error = 10000
no_weight_dist_error=0
for args in tqdm(acceptable[['model_name','distance_frac_step']].drop_duplicates().values.tolist()):
    temp = df[
        (df["model_name"] == args[0]) &
        (df["distance_frac_step"] == args[1]) &
        (df["clean_data"]==selected_args_ratio[0]) &
        (df["optimized_mode_ratio"]==selected_args_ratio[1]) &
        (df["butter_Wn"]==selected_args_ratio[2])
    ]
    if len(temp) == 0:
        continue
    mean_dist_error = (temp["dist_error"] * temp["weight"]).sum() / temp["weight"].sum()
    if mean_dist_error < min_mean_dist_error:
        min_mean_dist_error = mean_dist_error
        selected_args = args
        no_weight_dist_error = temp["dist_error"].mean()

100%|██████████| 2/2 [00:00<00:00, 249.72it/s]


In [7]:
print('Best args: '+json.dumps({
    "model_name": selected_args[0],
    "distance_frac_step": selected_args[1],
    "clean_data": selected_args_ratio[0],
    "optimized_mode_ratio": selected_args_ratio[1],
    "butter_Wn": selected_args_ratio[2]
    }, indent=4))
print('Mean dist_error (with weight): '+ str(min_mean_dist_error))
print('Mean dist_error (without weight): '+ str(no_weight_dist_error))
data=df[
    (df["model_name"] == selected_args[0]) &
    (df["distance_frac_step"] == selected_args[1]) &
    (df["clean_data"]==selected_args_ratio[0]) &
    (df["optimized_mode_ratio"]==selected_args_ratio[1]) &
    (df["butter_Wn"]==selected_args_ratio[2])
][["dataset", "dist_error", "dir_error", "dir_ratio", "weight"]].sort_values('dir_ratio', ascending=False).reset_index(drop=True)
print('Mean dir_ratio (with weight): '+ str((data["dir_ratio"]*data["weight"]).sum()/data["weight"].sum()))
print('Mean dir_ratio (without weight): '+ str(data["dir_ratio"].mean()))
data

Best args: {
    "model_name": "SVR",
    "distance_frac_step": 4,
    "clean_data": 4.0,
    "optimized_mode_ratio": 0.95,
    "butter_Wn": 0.0035
}
Mean dist_error (with weight): 64.4871270136612
Mean dist_error (without weight): 93.00095376508295
Mean dir_ratio (with weight): 0.798321924289776
Mean dir_ratio (without weight): 0.6926052989458058


,dataset,dist_error,dir_error,dir_ratio,weight
0,Pocket-Walk/Pocket-Walk-09-001,73.460831,5.704716,0.938144,1
1,Hand-Walk/Hand-Walk-02-001,14.711121,7.143046,0.928889,1
2,../test_case0,30.032920,8.662983,0.926063,25
3,Hand-Walk/Hand-Walk-09-001,35.989921,8.284083,0.922078,1
4,Hand-Walk/Hand-Walk-02-008,22.224224,8.734746,0.855037,1
5,Hand-Walk/Hand-Walk-15-004,89.331216,10.718334,0.821359,1
6,Hand-Walk/Hand-Walk-03-011,145.537911,8.963318,0.805556,1
7,Hand-Walk/Hand-Walk-02-009,32.103093,11.525013,0.793774,1
8,Pocket-Walk/Pocket-Walk-03-008,63.014326,11.079511,0.767225,1
9,Hand-Walk/Hand-Walk-02-003,42.985622,11.798218,0.764045,1
